# Using functional programming in Python like a boss: Generators, Iterators and Decorators

<img src="figures/simple_man.jpg\" style="display:block;margin:auto;width:80%;"/>

# Features of functions
1. First-class functions are objects and thus:
    * Can be assigned to variables
    * Can be stored in data structures
    * Can be used as parameters
    * Can be used as a return value
2. Higher order functions:
    * Accept a function as an argument and/or return a function as a value.
    * Create composite functions from simpler functions.
    * Modify the behavior of existing functions.
3. Pure functions:
   * Do not depend on hidden state, or equivalently depend only on their input.
   * Evaluation of the function does not cause side effects


In [1]:
# A pure function
def my_min(x, y):
    if x < y:
        return x
    else:
        return y


# An impure function
# 1) Depends on global variable, 2) Changes its input
exponent = 2

def my_powers(L):
    for i in range(len(L)):
        L[i] = L[i]**exponent
    return L

# What can act as a function in Python?
1. A function object, created with the `def` statement.
2. A `lambda` anonymous function, restricted to an expression in single line.
3. An instance of a class implementing the `__call__` function.

In [2]:
def min_def(x, y):
    return x if x < y else y

min_lambda = lambda x, y: x if x < y else y

class MinClass:
    def __call__(self, x, y):
        return x if x < y else y

min_class = MinClass()
print(min_def(2,3) == min_lambda(2, 3) == min_class(2,3))

True


# Common gotchas 1: Mutable Default Arguments

* Python's default arguments are evaluated once when the function is **defined** and **not each time the function is called** (like say, in Ruby).
* If you use a mutable default argument and mutate it, you will and have mutated that object for all future calls to the function as well.
* Sometimes you can specifically "exploit" (read: use as intended) this behavior to maintain state between calls of a function. This is often done when writing a caching function.

In [3]:
def append_to(element, to=[]):
    to.append(element)
    return to

my_list = append_to(12)
print("my_list:", my_list)
my_other_list = append_to(42)
print("my_other_list:", my_other_list)

def append_to2(element, to=None):
    if to is None:
        to = []
    to.append(element)
    return to

my_list2 = append_to2(12)
print("my_list2:", my_list2)
my_other_list2 = append_to2(42)
print("my_other_list2:", my_other_list2)

my_list: [12]
my_other_list: [12, 42]
my_list2: [12]
my_other_list2: [42]


# Common gotchas 2: Late Binding Closures

1. A closure occurs when a function has access to a local variable from an enclosing scope that has finished its execution.
2. Python's closures are *late binding*.
3. Values of variables used in closures are looked up at the time the inner function is **called** and **not when it is defined**.

In [4]:
def create_multipliers():
    multipliers = []

    for i in range(5):
        def multiplier(x):
            return i * x
        multipliers.append(multiplier)

    return multipliers

for multiplier in create_multipliers():
    print(multiplier(2))

8
8
8
8
8


# Higher order functions and decorators

* Python functions are objects.
* Can be defined in functions.
* Can be assigned to variables.
* Can be used as function parameters or returned from functions.
* Decorators are syntactic sugar for higher order functions.

<img src="figures/brace_yourselves.jpg" style="display:block;margin:auto;width:80%;"/>

In [12]:
# Higher order functions

def makebold(fn):
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped

def hello():
    return "hello world"

print(hello())
hello = makebold(hello)
print(hello())

hello world
<b>hello world</b>


In [6]:
# Decorated function with *args and **kewargs

def makebold(fn):
    def wrapped(*args, **kwargs):
        return "<b>" + fn(*args, **kwargs) + "</b>"
    return wrapped

@makebold  # hello = makebold(hello)
def hello(*args, **kwargs):
    return "Hello. args: {}, kwargs: {}".format(args, kwargs)

print(hello('world', 'pythess', where='soho'))

<b>Hello. args: ('world', 'pythess'), kwargs: {'where': 'soho'}</b>


In [7]:
# Decorators can be combined

def makeitalic(fn):
    def wrapped(*args, **kwargs):
        return "<i>" + fn(*args, **kwargs) + "</i>"
    return wrapped

def makebold(fn):
    def wrapped(*args, **kwargs):
        return "<b>" + fn(*args, **kwargs) + "</b>"
    return wrapped

@makeitalic
@makebold  # hello = makeitalic(makebold(hello))
def hello(*args, **kwargs):
    return "Hello. args: {}, kwargs: {}".format(args, kwargs)

print(hello('world', 'pythess', where='soho'))

<i><b>Hello. args: ('world', 'pythess'), kwargs: {'where': 'soho'}</b></i>


In [8]:
# Decorators can be instances of callable classes

class BoldMaker:
    def __init__(self, fn):
        self.fn = fn
    def __call__(self, *args, **kwargs):
        return "<b>" + self.fn(*args, **kwargs) + "</b>"

@BoldMaker  # hello = Bookmaker(hello)
def hello(*args, **kwargs):
    return "Hello. args: {}, kwargs: {}".format(args, kwargs)

# hello.__call__(*args, **kwargs)
print(hello('world', 'pythess', where='soho'))

<b>Hello. args: ('world', 'pythess'), kwargs: {'where': 'soho'}</b>


In [9]:
# Decorators can take arguments

def enclose_in_tags(opening_tag, closing_tag):  # returns a decorator
    def make_with_tags(fn):  # returns a decorated function
        def wrapped():  # the function to be decorated (modified)
            return opening_tag + fn() + closing_tag
        return wrapped
    return make_with_tags

# decorator function make_with_tags with the arguments in closure
heading_decorator = enclose_in_tags('<h1>', '</h1>')
paragraph_decorator = enclose_in_tags('<p>', '</p>')

def hello():
    return "hello world"

h1_hello = heading_decorator(hello)
p_hello = paragraph_decorator(hello)
h1_p_hello = heading_decorator(paragraph_decorator(hello))
print(h1_hello())
print(p_hello())
print(h1_p_hello())

<h1>hello world</h1>
<p>hello world</p>
<h1><p>hello world</p></h1>


In [22]:
# Decorators with arguments combined

def enclose_in_tags(opening_tag, closing_tag):
    def make_with_tags(fn):
        def wrapped():
            return opening_tag + fn() + closing_tag
        return wrapped
    return make_with_tags

# hello = enclose_in_tags('<h1>', '</h1>')(hello)
@enclose_in_tags('<h1>', '</h1>') 
def hello():
    return "hello world"

print(hello())

# hello = enclose_in_tags('<p>', '</p>')(hello)
@enclose_in_tags('<p>', '</p>')
def hello():
    return "hello world"

print(hello())

# enclose_in_tags('<h1>', '</h1>')(enclose_in_tags('<p>', '</p>')(hello))
@enclose_in_tags('<h1>', '</h1>')
@enclose_in_tags('<p>', '</p>')
def hello():
    return "hello world"

print(hello())

<h1>hello world</h1>
<p>hello world</p>
<h1><p>hello world</p></h1>


In [23]:
# Decorators with arguments as instances of callable classes

class TagEncloser:
    def __init__(self, opening_tag, closing_tag):
        self.opening_tag = opening_tag
        self.closing_tag = closing_tag
    def __call__(self, fn):
        def wrapped():
            return self.opening_tag + fn() + self.closing_tag
        return wrapped

tag_h1 = TagEncloser('<h1>', '</h1>')
tag_p = TagEncloser('<p>', '</p>')

@tag_h1
@tag_p
def hello():  # hello = tag_h1(tag_p(hello))
    return "hello world"

print(hello())

<h1><p>hello world</p></h1>


# Iterables and Iterators

1. **Iteration** is a general term for taking each item of something, one after another. Any time you use a loop, explicit or implicit, to go over a group of items, that is iteration.
2. An **iterable** is an object that either:
    * Has an `__iter__` method which returns an **iterator** for that object.
    * Defines a `__getitem__` method that can take sequential indexes starting from zero (and raises an `IndexError` when the indexes are no longer valid). So an iterable is an object that you can get an iterator from.
3. An **iterator** is a stateful helper object which defines a  `__next__` method and will produce the next value when you call ``next()`` on it. If there are no further items, it raises the `StopIteration` exception.

In [5]:
x = [1, 2, 3]  # this is an iterable
y = iter(x)  # an iterator of this iterable
z = iter(x)  # another iterator of this iterable
print(next(y))
print(next(y))
print(next(z))
print(type(x))
print(type(y))

1
2
1
<class 'list'>
<class 'list_iterator'>


* Here, `x` is the iter**able**, while `y` and `z` are two individual instances of an iterat**or**, producing values from the iterable x. Both `y` and `z` hold state, as you can see from the example. In this example, `x` is a data structure (a list), but that is not a requirement.
* Often, for pragmatic reasons, iterable classes will implement both `__iter__()` and `__next__()` in the same class, and have `__iter__()` return `self`, which makes the class **both an iterable and its own iterator**. It is perfectly fine to return a different object as the iterator, though.

When you write:
```python
x = [1, 2, 3]
for elem in x:
    ...
```
This is what actually happens:
<img src="figures/iterable_vs_iterator.png" style="display:block;margin:auto;width:80%;"/>

In [32]:
s = 'cat'  # s is an ITERABLE
           # s is a str object that is immutable
           # s has no state
           # s has a __getitem__() method 

t = iter(s)    # t is an ITERATOR
               # t has state (it starts by pointing at the "c"
               # t has a next() method and an __iter__() method
try:
    print(next(t))        # the next() function returns the next value and advances the state
    print(next(t))        # the next() function returns the next value and advances
    print(next(t))        # the next() function returns the next value and advances
    print(next(t))        # next() raises StopIteration to signal that iteration is complete
except StopIteration as e:
    print("StopIteration raised")

c
a
t
StopIteration raised


In [35]:
# iterator that produces an arbitrary number of the Fibonacci series

from itertools import islice

class FibonacciIterator:
    def __init__(self):
        self.prev = 0
        self.curr = 1
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.prev, self.curr = self.curr, self.prev + self.curr
        return self.prev
f = FibonacciIterator()
print(list(islice(f, 0, 10)))
f2 = FibonacciIterator()
print([next(f2) for _i in range(10)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


# Generators


# References

1. [K. Reitz - The Hitchhiker's Guide to Python](http://docs.python-guide.org/en/latest)
* [PyToolz API Documentation](http://toolz.readthedocs.io/en/latest)
* [StackOverflow - How to make a chain of function decorators in Python?](http://stackoverflow.com/questions/739654/how-to-make-a-chain-of-function-decorators-in-python)
* [StackOverflow - What exactly are Python's iterator, iterable, and iteration protocols?](http://stackoverflow.com/questions/9884132/what-exactly-are-pythons-iterator-iterable-and-iteration-protocols)
* [Bruce Eckel - Decorators I: Introduction to Python Decorators](http://www.artima.com/weblogs/viewpost.jsp?thread=240808)
* [Bruce Eckel - Python Decorators II: Decorator Arguments](http://www.artima.com/weblogs/viewpost.jsp?thread=240845)
* [V. Driessen - Iterables vs. Iterators vs. Generators](http://nvie.com/posts/iterators-vs-generators)
* D. Mertz - Functional Programming in Python, O' Reilly (2015)
* S. Lott - Functional Python Programming, Packt Publishing (2015)